In [ ]:
import pandas as pd
import os
from pathlib import Path

# function duplicated here because Python path is strange when run in Dagster
DATA_DIR = "data"  # if no env var, default directory where collections directory lives


def get_cartilage_thickness_runs_file_path():
    root_dir = os.getenv("FILE_STORAGE_ROOT", DATA_DIR)
    oai_collection = Path(root_dir) / "collections" / "oai"
    runs_file = oai_collection / "cartilage_thickness_runs.parquet"
    return runs_file

In [ ]:
runs_file = get_cartilage_thickness_runs_file_path()
runs = pd.read_parquet(runs_file)

unique_computed_files_dir = runs.drop_duplicates(
    subset="computed_files_dir", keep="last"
)["computed_files_dir"].tolist()
unique_parents = list(
    dict.fromkeys(reversed([Path(dir).parent for dir in unique_computed_files_dir]))
)
# child directories starting with most recent
parent_child_dirs = [
    (
        parent,
        sorted(list(parent.glob("*")), key=lambda x: x.stat().st_mtime, reverse=True),
    )
    for parent in unique_parents
]
studies = parent_child_dirs[:20]
image_file_names = ["FC_thickness.png", "TC_thickness.png"]

### Show thickness images for each code version of each study

Last 20 processed studies, starting with most recent.

In [ ]:
from IPython.display import display, HTML
import base64

IMAGE_SIZE = 300  # Thumbnail size in pixels

# CSS and JavaScript for modal functionality
modal_js_css = """
<style>
/* The Modal (background) */
.modal {
  display: none;
  position: fixed;
  z-index: 1000;
  padding-top: 60px;
  left: 0;
  top: 0;
  width: 100%;
  height: 100%;
  overflow: auto;
  background-color: rgba(0,0,0,0.9);
}

/* Modal Content (Image) */
.modal-content {
  margin: auto;
  display: block;
  max-width: 90%;
  max-height: 90%;
}

/* Caption of Modal Image */
#caption {
  margin: auto;
  display: block;
  text-align: center;
  color: #ccc;
  padding: 10px 0;
}

/* Close Button */
.close {
  position: absolute;
  top: 20px;
  right: 35px;
  color: #fff;
  font-size: 40px;
  font-weight: bold;
  cursor: pointer;
}

.close:hover,
.close:focus {
  color: #bbb;
}

</style>

<script>
// Modal functionality
function initModal() {
  const modal = document.getElementById("myModal");
  const modalImg = document.getElementById("img01");
  const captionText = document.getElementById("caption");
  const closeButton = document.getElementsByClassName("close")[0];

  // Close modal when clicking on the "x"
  console.log(closeButton);
  closeButton.addEventListener("click", () => {
    modal.style.display = "none";
  });

  // Close modal when clicking anywhere
  modal.addEventListener("click", (event) => {
      modal.style.display = "none";
  });
}

// Show modal with full-sized image
function showModal(src, caption) {
  const modal = document.getElementById("myModal");
  const modalImg = document.getElementById("img01");
  const captionText = document.getElementById("caption");
  modal.style.display = "block";
  modalImg.src = src;
  captionText.innerHTML = caption;
}

initModal()
</script>
"""

# Modal HTML (included once)
modal_div = """
<div id="myModal" class="modal">
  <span class="close">&times;</span>
  <img class="modal-content" id="img01">
  <div id="caption"></div>
</div>
"""

html_content = modal_js_css + modal_div

for parent, child_dirs in studies:
    html_content += f"<h3>{parent}</h3><div style='display: flex; flex-wrap: wrap;'>"
    for child_dir in child_dirs:
        for image_file_name in image_file_names:
            image_path = child_dir / image_file_name
            if image_path.exists():
                with open(image_path, "rb") as img_file:
                    data = base64.b64encode(img_file.read()).decode()
                thumbnail_src = f"data:image/png;base64,{data}"
                caption = f"{child_dir.name} - {image_file_name}"
                html_content += f"""
                <div style='margin: 10px; text-align: center;'>
                    <img src="{thumbnail_src}" style='max-width: {IMAGE_SIZE}px; max-height: {IMAGE_SIZE}px; cursor: pointer;' onclick="showModal('{thumbnail_src}', '{caption}')"/><br>
                    {caption}
                </div>
                """
            else:
                html_content += f"""
                <div style='margin: 10px; text-align: center; width: {IMAGE_SIZE}px; height: {IMAGE_SIZE}px; display: flex; align-items: center; justify-content: center; border: 1px solid #ccc;'>
                    <span>{child_dir.name} {image_file_name} not found</span>
                </div>
                """
    html_content += "</div>"

# Display the HTML content
display(HTML(html_content))